In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import scipy.io
import scipy.sparse.linalg as sp_linalg
import pandas as pd
import networkx as nx
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
#procesar los labels correspondientes al grafo, extrae los valores únicos y la codificación one hot
def process_labels(lab):
    unique = np.unique(lab.flatten())
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(unique.reshape(-1, 1))
    oneHot = enc.transform(lab.reshape(-1, 1)).toarray()
    return unique, oneHot

In [3]:
#Ecuacion 3.2: calcula la autocovarianza de matrices para un camino aleatorio en un momento t
#t: momento a calcular
#P: matriz diagonal de distribución estacionaria
#p: vector de distribución estacionaria
#Me: matriz de markov
def ro(t, P, p, Me):
    return P * np.linalg.matrix_power(Me, t) - pi.transpose() * pi

In [4]:
#Ecuacion 3.3: covarianza del atributo v para los primeros t tiempos determinados por lag 
#v: atributos de cada nodo (arreglo de números)
#cova: arreglo de covarianzas para los primeros t tiempos
#lag: numero de tiempos a calcular
def attr_cov(v, cova, lag):
    cov = []
    for t in range(lag+1):
        cov.append((v.transpose() * cova[t] * v).tolist()[0][0])
    return cov

In [5]:
#Ecuacion 3.6: covarianza de variable categorica para los primeros t tiempos determinados por lag
#Me: matriz de markov
#p: vector de distribución estacionaria
#P: matriz diagonal de distribución estacionaria
#Ha: matriz one hot de la variable categorica
#lag: numero de tiempos a calcular
def covarianza(Me, p, P, Ha, lag): 
    # cov son las matrices de covarianza
    # aut son las matrices de autocovarianza
    # tr son las trazas de aut
    cov = []
    aut = []
    tr = []
    for t in range(lag +1):
        cov.append(ro(t, P, p, Me))
        aut.append(Ha.transpose() * cov[-1] * Ha)
        tr.append(np.trace(aut[-1]))
    return cov, aut, tr

In [6]:
#Calcula el promedio de los labels de los nodos
#Ha: matriz one hot de la variable categorica
#p: vector de distribución estacionaria
def avg_feats(Ha, p):
    avg = []
    for i in range(len(Ha[0])):
        avg.append(p * np.asmatrix(Ha[:,i]).transpose())
    return avg

In [7]:
titles = ['ID', 'pagerank', 'eigenvec_2nd_M', 'eigenvec_3rd_M', 'node_attribs', 'avg_node_labels', 'n_edges', 'avg_pi', 'avg_inv_pi', 'avg_eigenvec_2nd', 'avg_eigenvec_3rd']
#nombre del dataset
name = 'mutag'
#si utilizar los labels de los nodos
node_labels = True
#numero de tiempos a calcular
l = 3
#numero de vectors propios a utilizar
num_eig = 3

In [8]:
#carga del dataset
dataset = scipy.io.loadmat('datasets/'+name+'.mat')
names = list(dataset.keys())[3:]

In [9]:
data = {}
df = None
Graphs = dataset[names[1]][0]
num_graphs = len(Graphs)
for i in range(num_graphs):
    print('procesando grafo en posicion', i)
    #labels de los nodos
    labels = Graphs[i][1][0][0][0].flatten()
    #total de nodos
    total = len(Graphs[i])
    #Construccion del grafo
    G = nx.from_numpy_matrix(Graphs[i][0])
    #A es la matriz de adjacencia del grafo
    A = nx.adjacency_matrix(G).todense()
    #d es el arreglo de grados del grafo
    d = A * np.ones((G.number_of_nodes(), 1))
    #D es la matriz de grados diagonales
    D = np.diag(d.transpose().tolist()[0])
    #M es la matriz de markov del grafo
    M = np.linalg.inv(D) * A
    # m es la cantidad de arcos del grafo
    m = len(G.edges)
    #pi es el vector de distribución estacionaria
    pi = d.transpose() / (2*m)
    #PI es la matriz diagonal de distribución estacionaria
    PI = np.diag(pi.tolist()[0])
    #calculo de los primeros eigenvalues y egenvectors de mayor a menor magnitud
    evalues, evectors = sp_linalg.eigs(M.transpose(), k = num_eig, which = 'LM')
    #feature1: multi-hop assortativities of node IDs, setting H = I in Eq. (3.6) for 0, 1, ... , t hops
    #H es la matriz identidad del tamaño del grafo
    H = np.identity(len(d))
    cov, aut, res1 = covarianza(M, pi, PI, H, l)
    #feature2: average of first p dominant left eigenvector of M, that is {πv : v is a dominant eigenvector of M}
    res2 = attr_cov(pi.transpose(), cov, l)
    #feature3: multi-hop assortativities of the first p dominant left eigenvectors of M (Eq. 3.3) for 0, 1, ... , t hops
    #segundo eigenvector
    pi2 = np.asmatrix(evectors[:,1]).transpose()
    #normalizacion del segundo eigenvector
    h2 = pi2/np.linalg.norm(pi2, 1)
    res3 = attr_cov(h2, cov, l)
    #tercer eigenvector
    pi3 = np.asmatrix(evectors[:,2]).transpose()
    #normalizacion del tercer eigenvector
    h3 = pi3/np.linalg.norm(pi3, 1)
    res4 = attr_cov(h3, cov, l)
    if node_labels:
        #extraccion de labels del grafo
        unique, categorias = process_labels(labels)
        cov, aut, tr = covarianza(M, pi, PI, categorias, l)
        #feature4: (If available) average of categorical metadata node attributes: {πhi : H = [h1, h2, ..hk ], 1 ≤ i ≤ k}
        res5 = np.array([np.diag(i) for i in aut]).flatten()
        #feature5: (If available) multi-hop assortativities of categorical metadata node attributes (Eq. 3.4) for 0, 1, ... , t hops
        res6 = avg_feats(categorias, pi)
    #feature6: number of nodes
    res9 = G.number_of_nodes()
    #feature7: number of edges
    res7 = G.number_of_edges()

    r = {
        0: res1,
        1: [i.real for i in res2],
        2: [i.real for i in res3],
        3: [i.real for i in res4]
    }
    if node_labels:
        r[4] = res5
        r[5] = [el.tolist()[0][0] for el in res6]
    r[6] = res7
    r[7] = (pi * pi.transpose()).tolist()[0][0]
    r[8] = res9
    r[9] = (pi * pi2).tolist()[0][0].real
    r[10] = (pi * pi3).tolist()[0][0].real
    
    for title in range(len(titles)):
        if title in r:
            if type(r[title]) != float and type(r[title]) != int :
                for j in range(len(r[title])):
                    key = titles[title] + '_' + str(j)
                    if not key in data:
                        data[key] = [0 for z in range(i)]
                    curr_size = len(data[key])
                    if  curr_size < i - 1:
                        data[key] = data[key] + [0 for z in range(i- curr_size)]
                    data[key].append(r[title][j])
            else:
                key = titles[title]
                if not key in data:
                    data[key] = []
                data[key].append(r[title])
for key in data.keys():
    curr_size = len(data[key])
    if  curr_size < num_graphs:
        data[key] = data[key] + [0 for z in range(num_graphs - curr_size)]
    print(curr_size, len(data[key]))
df = pd.DataFrame(data)
df

procesando grafo en posicion 0
procesando grafo en posicion 1
procesando grafo en posicion 2
procesando grafo en posicion 3
procesando grafo en posicion 4
procesando grafo en posicion 5
procesando grafo en posicion 6
procesando grafo en posicion 7
procesando grafo en posicion 8
procesando grafo en posicion 9
procesando grafo en posicion 10
procesando grafo en posicion 11
procesando grafo en posicion 12
procesando grafo en posicion 13
procesando grafo en posicion 14
procesando grafo en posicion 15
procesando grafo en posicion 16
procesando grafo en posicion 17
procesando grafo en posicion 18
procesando grafo en posicion 19
procesando grafo en posicion 20
procesando grafo en posicion 21
procesando grafo en posicion 22
procesando grafo en posicion 23
procesando grafo en posicion 24
procesando grafo en posicion 25
procesando grafo en posicion 26
procesando grafo en posicion 27
procesando grafo en posicion 28
procesando grafo en posicion 29
procesando grafo en posicion 30
procesando grafo e

,ID_0,ID_1,ID_2,ID_3,pagerank_0,pagerank_1,pagerank_2,pagerank_3,eigenvec_2nd_M_0,eigenvec_2nd_M_1,...,node_attribs_12,node_attribs_13,node_attribs_14,node_attribs_15,avg_node_labels_3,node_attribs_16,node_attribs_17,node_attribs_18,node_attribs_19,avg_node_labels_4
0,0.953361,-0.046639,0.379287,-0.046639,0.000111,0.000009,0.000039,-0.000003,0.002286,-0.002185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.954719,-0.045281,0.419005,-0.045281,0.000182,-0.000069,0.000083,-0.000033,0.002222,-0.001841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.943182,-0.056818,0.375000,-0.056818,0.000176,-0.000035,0.000070,-0.000026,0.000176,-0.000035,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.953361,-0.046639,0.379287,-0.046639,0.000111,0.000009,0.000043,-0.000001,0.002286,-0.002185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.936288,-0.063712,0.383657,-0.063712,0.000242,-0.000013,0.000096,-0.000020,0.004299,-0.004048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.896694,-0.103306,0.396694,-0.103306,0.000973,-0.000342,0.000285,-0.000143,0.015064,0.010308,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184,0.902778,-0.097222,0.402778,-0.097222,0.000965,-0.000482,0.000338,-0.000249,0.010417,-0.008970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.913265,-0.086735,0.377551,-0.086735,0.000586,-0.000143,0.000221,-0.000141,0.000586,-0.000143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186,0.915816,-0.084184,0.380102,-0.084184,0.000475,-0.000072,0.000171,-0.000092,0.007555,-0.007022,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df.to_csv('features/' + name + '_features_py.csv', sep=',', header=True)